In [1]:
import os 
os.chdir("../")
%pwd 

'd:\\Programming\\ML\\End-to-End\\CV\\End-to-End-PlantVillage'

In [2]:
from dataclasses import dataclass 
from pathlib import Path 

@dataclass 
class EvaluationSchema:
    track_url: str
    model_path: Path
    training_data: Path 
    all_params: dict 
    params_image_size: list 
    params_batch_size: int 
    


In [3]:
from src.PlantVillage.constants import PARAMS_FILE_PATH, CONFIG_FILE_PATH 
from src.PlantVillage.utils import read_yaml, create_directories 


class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
        
    def get_evaluation_config(self) -> EvaluationSchema:
        training_data_path = Path(os.path.join(self.config.data_ingestion.local_path, "PlantVillage"))
        
        
        return EvaluationSchema(
            track_url= self.config.evaluation.track_url,
            model_path= self.config.training.trained_model_path,
            training_data= training_data_path,
            all_params=  self.params,
            params_image_size= self.params.IMAGE_SIZE,
            params_batch_size= self.params.BATCH_SIZE,
        )

In [6]:
import tensorflow as tf
from src.PlantVillage.utils import save_json
import mlflow
import dagshub
from urllib.parse import urlparse


class Evaluation:
    def __init__(self, config: EvaluationSchema):
        self.config = config 
        
    def _valid_generator(self):
        
        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split = 0.2
        )
        
        dataflow_kwargs = dict(
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size,
            interpolation = "bilinear"
        )
        
        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )
        
        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset= "validation",
            shuffle= False,
            **dataflow_kwargs
        )
        
    def evaluation(self):
        self.model = tf.keras.models.load_model(self.config.model_path)
        self._valid_generator() 
        self.score = self.model.evaluate(self.valid_generator)
        self.save_score(self.score) 
        
    @staticmethod
    def save_score(score):
        scores = {"loss": score[0], "accuracy": score[1]}
        save_json(path= Path("scores.json"), data= scores)
        
        
    def log_into_mlflow(self):
        dagshub.init(repo_owner='youssefBedeer', repo_name='End-to-End-PlantVillage', mlflow=True)

        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            
            

In [7]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2026-01-26 22:33:58,666][INFO][__init__] -> yaml file: config\config.yaml loaded Successfully.
[2026-01-26 22:33:58,669][INFO][__init__] -> yaml file: params.yaml loaded Successfully.
[2026-01-26 22:33:58,671][INFO][__init__] -> Directory created Successfully at: artifacts
Found 4122 images belonging to 15 classes.
258/258 ━━━━━━━━━━━━━━━━━━━━ 322s 1s/step - accuracy: 0.7145 - loss: 7.4850
[2026-01-26 22:39:21,160][INFO][__init__] -> json file saved at: scores.json
[2026-01-26 22:39:22,284][INFO][_client] -> HTTP Request: GET https://dagshub.com/api/v1/user "HTTP/1.1 200 OK"


Accessing as youssefBedeer

[2026-01-26 22:39:22,291][INFO][helpers] -> Accessing as youssefBedeer
[2026-01-26 22:39:23,344][INFO][_client] -> HTTP Request: GET https://dagshub.com/api/v1/repos/youssefBedeer/End-to-End-PlantVillage "HTTP/1.1 200 OK"
[2026-01-26 22:39:24,648][INFO][_client] -> HTTP Request: GET https://dagshub.com/api/v1/user "HTTP/1.1 200 OK"


Initialized MLflow to track repo "youssefBedeer/End-to-End-PlantVillage"

[2026-01-26 22:39:24,651][INFO][helpers] -> Initialized MLflow to track repo "youssefBedeer/End-to-End-PlantVillage"


Repository youssefBedeer/End-to-End-PlantVillage initialized!

[2026-01-26 22:39:24,654][INFO][helpers] -> Repository youssefBedeer/End-to-End-PlantVillage initialized!


2026/01/26 22:39:27 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2026/01/26 22:39:29 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
Successfully registered model 'VGG16Model'.
2026/01/26 22:42:57 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: VGG16Model, version 1
Created version '1' of model 'VGG16Model'.


🏃 View run dapper-cub-137 at: https://dagshub.com/youssefBedeer/End-to-End-PlantVillage.mlflow/#/experiments/0/runs/26b60d1bcf8a44cc8b0a1c567aa64384
🧪 View experiment at: https://dagshub.com/youssefBedeer/End-to-End-PlantVillage.mlflow/#/experiments/0
